# Решение контеста https://sascompetitions.ru/competitions/forecasting-the-probability-of-non-repayment-of-loan

### Амир Мирас Сабыргалиулы

In [1]:
import numpy as np
import pandas as pd
import pandas as pd
from xgboost import XGBClassifier
from datetime import datetime
from tqdm import tqdm_notebook
import re
from sklearn.model_selection import cross_val_score, cross_val_predict
import  matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

%matplotlib inline

### Вспомогательные патчи

In [2]:
def df_only(df, *patterns):
    columns = set()
    for pattern in patterns:
        columns |= set([x for x in df.columns if re.search("^{}".format(pattern), x)])
    return df[list(columns)]

def df_omit(df, *patterns):
    columns = set(df.columns)
    for pattern in patterns:
        columns &= set([x for x in df.columns if not bool(re.search("^{}".format(pattern), x))])
    return df[list(columns)]


pd.DataFrame.only = df_only
pd.DataFrame.omit = df_omit

### Загрузка данных и препроцессинг

In [3]:
%%time
parse_dates = ['DTIME_CREDIT', 'DTIME_CREDIT_ENDDATE',
    'DTIME_CREDIT_ENDDATE_FACT', 'DTIME_CREDIT_UPDATE']

data = pd.read_csv('sas_test.csv',
    parse_dates=parse_dates, dayfirst=True, infer_datetime_format=True).append(
    pd.read_csv('sas_train.csv',
    parse_dates=parse_dates, dayfirst=True, infer_datetime_format=True))
data.reset_index(drop=True, inplace=True)

CPU times: user 31.4 s, sys: 732 ms, total: 32.1 s
Wall time: 32.9 s


In [4]:
%%time
sorts = data['DTIME_CREDIT'].sort_values()
bad_idxs = sorts.index[:3]
true_idx = sorts.index[3]
data.loc[bad_idxs, 'DTIME_CREDIT'] = data.loc[true_idx, 'DTIME_CREDIT']

sorts = data['DTIME_CREDIT_UPDATE'].sort_values(ascending=False)
bad_idxs = sorts.index[:2]
true_idx = sorts.index[3]
data.loc[bad_idxs, 'DTIME_CREDIT_UPDATE'] = data.loc[true_idx, 'DTIME_CREDIT_UPDATE']

CPU times: user 980 ms, sys: 60 ms, total: 1.04 s
Wall time: 1.04 s


In [5]:
%%time
data['SK_DATE_DECISION'] = data['SK_DATE_DECISION'].apply(
        lambda x: datetime(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:]))
)

CPU times: user 4.87 s, sys: 88 ms, total: 4.96 s
Wall time: 4.96 s


### Тэгирование столбцов для удобства

In [6]:
rename_columns = {
    'AMT_ANNUITY': 'AMT:ANNUITY',
    'AMT_CREDIT_MAX_OVERDUE': 'AMT:CREDIT_MAX_OVERDUE',
    'AMT_CREDIT_SUM': 'AMT:CREDIT_SUM',
    'AMT_CREDIT_SUM_DEBT': 'AMT:CREDIT_SUM_DEBT',
    'AMT_CREDIT_SUM_LIMIT': 'AMT:CREDIT_SUM_LIMIT',
    'AMT_CREDIT_SUM_OVERDUE': 'AMT:CREDIT_SUM_OVERDUE',
    'AMT_REQ_SOURCE_DAY': 'AMT:REQ_SOURCE_DAY',
    'AMT_REQ_SOURCE_HOUR': 'AMT:REQ_SOURCE_HOUR',
    'AMT_REQ_SOURCE_MON': 'AMT:REQ_SOURCE_MON',
    'AMT_REQ_SOURCE_QRT': 'AMT:REQ_SOURCE_QRT',
    'AMT_REQ_SOURCE_WEEK': 'AMT:REQ_SOURCE_WEEK',
    'AMT_REQ_SOURCE_YEAR': 'AMT:REQ_SOURCE_YEAR',
    'CNT_CREDIT_PROLONG': 'CNT:CREDIT_PROLONG',
    'CREDIT_ACTIVE': 'CAT:CREDIT_ACTIVE',
    'CREDIT_COLLATERAL': 'CAT:CREDIT_COLLATERAL',
    'CREDIT_CURRENCY': 'CAT:CREDIT_CURRENCY',
    'CREDIT_DAY_OVERDUE': 'CREDIT_DAY_OVERDUE',
    'CREDIT_DELAY30': 'CREDIT_DELAY30',
    'CREDIT_DELAY5': 'CREDIT_DELAY5',
    'CREDIT_DELAY60': 'CREDIT_DELAY60',
    'CREDIT_DELAY90': 'CREDIT_DELAY90',
    'CREDIT_DELAY_MORE': 'CREDIT_DELAY_MORE',
    'CREDIT_FACILITY': 'CAT:CREDIT_FACILITY',
    'CREDIT_SUM_TYPE': 'CAT:CREDIT_SUM_TYPE',
    'CREDIT_TYPE': 'CAT:CREDIT_TYPE',
    'DEF': 'DEF',
    'DTIME_CREDIT': 'DATE:DTIME_CREDIT',
    'DTIME_CREDIT_ENDDATE': 'DATE:DTIME_CREDIT_ENDDATE',
    'DTIME_CREDIT_ENDDATE_FACT': 'DATE:DTIME_CREDIT_ENDDATE_FACT',
    'DTIME_CREDIT_UPDATE': 'DATE:DTIME_CREDIT_UPDATE',
    'ID': 'ID',
    'NUM_SOURCE': 'CAT:NUM_SOURCE',
    'SK_DATE_DECISION': 'DATE:SK_DATE_DECISION',
    'TEXT_PAYMENT_DISCIPLINE': 'TEXT:PAYMENT_DISCIPLINE'
}

In [7]:
data = data.rename(columns=rename_columns)

### Фичи по датам

In [8]:
date_columns = data.only('DATE').columns
for i, col1 in tqdm_notebook(enumerate(date_columns), total=len(date_columns)):
    for j, col2 in enumerate(date_columns):
        if i < j:
            data["DELTA:{}_{}".format(col1, col2)] = (data[col1] - data[col2]).dt.days

HBox(children=(IntProgress(value=0, max=5), HTML(value='')))

### Фичи по тексту

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
data['TEXT:PAYMENT_DISCIPLINE_REVERSE'] = data['TEXT:PAYMENT_DISCIPLINE'].str[::-1]
masks = data['TEXT:PAYMENT_DISCIPLINE_REVERSE'].notnull()
vectorizer = CountVectorizer(ngram_range=(1, 2), analyzer='char')
data['TEXT:PAYMENT_DISCIPLINE_REVERSE_CLEAR'] = data.loc[
    masks, 'TEXT:PAYMENT_DISCIPLINE_REVERSE'].apply(lambda x: re.sub("[^0-9]", " ", x))

In [11]:
vectorizer.fit(data.loc[masks, 'TEXT:PAYMENT_DISCIPLINE_REVERSE_CLEAR'])

CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [12]:
fnames = vectorizer.get_feature_names()
use_names = [fnames.index(x) for x in fnames if not (len(x) == 2 and ' ' in x)]

In [13]:
text_vec = vectorizer.transform(data.loc[
    masks, 'TEXT:PAYMENT_DISCIPLINE_REVERSE_CLEAR']).toarray()

In [14]:
data_text = pd.DataFrame(
    text_vec[:, use_names], 
    columns=['COUNT:TEXT_{}'.format(x) for x in np.array(fnames)[use_names]],
    index=data[masks].index
)
data = data.join(data_text)

In [15]:
def weight_sum_inv(s):
    if s == s:
        return np.sum(1 / (np.array([x.start() for x in re.finditer('0', s)]) + 1))
    else:
        return np.nan

In [16]:
data['COUNT:TEXT_INV'] = data['TEXT:PAYMENT_DISCIPLINE_REVERSE_CLEAR'].apply(weight_sum_inv)

In [17]:
def weight_sum(x):
    s0 = str(x).count('0')
    s1 = str(x).count('1')
    s2 = str(x).count('2')
    s3 = str(x).count('3')
    s4 = str(x).count('4')
    s5 = str(x).count('5')
    return s0 + s1 * 3 + s3 * 5 + s4 * 7 + s5 * 9

In [18]:
data['COUNT:TEXT_LEN'] = data['TEXT:PAYMENT_DISCIPLINE'].str.len()
data['COUNT:TEXT_WEIGHT_SUM'] = data['TEXT:PAYMENT_DISCIPLINE'].apply(weight_sum)

### Фичи по времени

In [19]:
%%time
data['TIME:SK_DATE_WEEKDAY'] = data['DATE:SK_DATE_DECISION'].apply(lambda x: x.weekday())
data['TIME:SK_DATE_DAY'] = data['DATE:SK_DATE_DECISION'].apply(lambda x: x.day)

CPU times: user 15.8 s, sys: 228 ms, total: 16 s
Wall time: 15.9 s


In [27]:
%%time
data['TIME:DTIME_CREDIT_WEEKDAY'] = data['DATE:DTIME_CREDIT'].apply(lambda x: x.weekday())
data['TIME:DTIME_CREDIT_YEAR'] = data['DATE:DTIME_CREDIT'].apply(lambda x: x.year)
data['TIME:DTIME_CREDIT_MONTH'] = data['DATE:DTIME_CREDIT'].apply(lambda x: x.month)
data['TIME:DTIME_CREDIT_DAY'] = data['DATE:DTIME_CREDIT'].apply(lambda x: x.day)

CPU times: user 30.7 s, sys: 464 ms, total: 31.2 s
Wall time: 31.1 s


### Категориальные фичи

In [28]:
currency2code = {
    "rur": 0,
    "usd": 1,
    "eur": 2,
    "chf": 3
}

In [29]:
data['CAT:CREDIT_CURRENCY'] = data['CAT:CREDIT_CURRENCY'].map(currency2code)

In [32]:
counts = []
for col in tqdm_notebook(data.only("CAT", "TIME:DTIME_CREDIT_WEEKDAY", "TIME:DTIME_CREDIT_MONTH").columns):
    group_count = data.groupby(["ID", col]).size().reset_index()
    count = group_count.pivot(index="ID", columns=col, values=0).fillna(0.0)
    count.columns = ["COUNT:{}_{}".format(col, x) for x in count.columns]
    counts.append(count)

HBox(children=(IntProgress(value=0, max=9), HTML(value='')))

In [33]:
data_count = pd.concat(counts, axis=1).reset_index()
data = data.merge(data_count, on='ID', how='left')

In [34]:
%%time
lengths = data.groupby('ID').size()
labels = data.groupby('ID')['DEF'].first()
idxs = data.index
data.drop('DEF', axis=1, inplace=True)

CPU times: user 1.62 s, sys: 476 ms, total: 2.09 s
Wall time: 2.09 s


In [35]:
data_len = lengths.reset_index().rename(columns={0: 'COUNT:SIZE'})
data = data.merge(data_len, on='ID', how='left')

### Некоторые операции над другими фичами

In [36]:
data['OP:SUB_SUM_DEBT'] = data['AMT:CREDIT_SUM'] - data['AMT:CREDIT_SUM_DEBT']
data['OP:REL_SUM_DEBT'] = data['AMT:CREDIT_SUM_DEBT'] / (data['AMT:CREDIT_SUM'] + 1)
data['OP:REL_SUM_ANNUITY'] = data['AMT:CREDIT_SUM'] / (data['AMT:ANNUITY'] + 1)
data['OP:REL_SUM_DELTA_CREDIT'] = data['AMT:CREDIT_SUM'] / (
    data['DELTA:DATE:DTIME_CREDIT_ENDDATE_FACT_DATE:DTIME_CREDIT'] + 0.1)
data['OP:REL_SUM_DELTA_CREDIT_AMT_ANNUITY'] = data['AMT:ANNUITY'] - \
                                              data['OP:REL_SUM_DELTA_CREDIT']
data['OP:SUB_SUM_MAX_OVERDUE'] = data['AMT:CREDIT_SUM_OVERDUE'] - \
                                 data['AMT:CREDIT_MAX_OVERDUE']
data['OP:REL_DEBT_OVERDUE'] = data['AMT:CREDIT_SUM_DEBT'] / (
    data['AMT:CREDIT_SUM_OVERDUE'] + 2)
data['OP:SUB_ANNUITY_SUM_DELTA_CREDIT'] = data['AMT:ANNUITY'] - data['OP:REL_SUM_DELTA_CREDIT']
data['OP:REL_SUM_MAX_OVERDUE'] = data['AMT:CREDIT_SUM_OVERDUE'] - \
                                 data['AMT:CREDIT_MAX_OVERDUE']
data['OP:REL_DEBT_OVERDUE'] = data['AMT:CREDIT_SUM_DEBT'] / \
                             (data['AMT:CREDIT_SUM_OVERDUE'] + 2)

In [38]:
use_columns = ['COUNT', 'TIME', 
               'CREDIT_DELAY', 'AMT', 
               'CNT', 'OP','DELTA',
               'DELTA', 'CREDIT_DAY_OVERDUE',
               'CAT:CREDIT_COLLATERAL', 'ID', 
               'CAT:CREDIT_ACTIVE']

In [39]:
data_use = data.only(*use_columns)

### Агрегирование фичей как среднее, максимум, медиана, стандартное отклонение

In [41]:
%%time
grouped_mean = data_use.groupby('ID').mean()
grouped_active_mean = data_use[data_use['CAT:CREDIT_ACTIVE'] == 1].omit('CAT:CREDIT_ACTIVE').groupby('ID').mean()

grouped_max = data_use.groupby('ID').max()
grouped_active_max = data_use[data_use['CAT:CREDIT_ACTIVE'] == 1].omit('CAT:CREDIT_ACTIVE').groupby('ID').max()

grouped_median = data_use.groupby('ID').median()
grouped_active_median = data_use[data_use['CAT:CREDIT_ACTIVE'] == 1].omit('CAT:CREDIT_ACTIVE').groupby('ID').median()

grouped_std = data_use.groupby('ID').std()
grouped_active_std = data_use[data_use['CAT:CREDIT_ACTIVE'] == 1].omit('CAT:CREDIT_ACTIVE').groupby('ID').std()

grouped_mean.columns = ["MEAN:{}".format(col) for col in grouped_mean.columns]
grouped_active_mean.columns = ["ACTIVE_MEAN:{}".format(col) for col in grouped_active_mean.columns]

grouped_max.columns = ["MAX:{}".format(col) for col in grouped_max.columns]
grouped_active_max.columns = ["ACTIVE_MAX:{}".format(col) for col in grouped_active_max.columns]

grouped_median.columns = ["MEDIAN:{}".format(col) for col in grouped_median.columns]
grouped_active_median.columns = ["ACTIVE_MEDIAN:{}".format(col) for col in grouped_active_median.columns]

grouped_std.columns = ["STD:{}".format(col) for col in grouped_std.columns]
grouped_active_std.columns = ["ACTIVE_STD:{}".format(col) for col in grouped_active_std.columns]

CPU times: user 19.6 s, sys: 2.23 s, total: 21.8 s
Wall time: 21.8 s


In [42]:
grouped_data = grouped_mean.merge(
    grouped_active_mean, left_index=True, right_index=True, how='left'
).merge(
    grouped_max, left_index=True, right_index=True, how='left'
).merge(
    grouped_active_max, left_index=True, right_index=True, how='left'
).merge(
    grouped_std, left_index=True, right_index=True, how='left'
).merge(
    grouped_active_std, left_index=True, right_index=True, how='left'
).merge(
    grouped_median, left_index=True, right_index=True, how='left'
).merge(
    grouped_active_median, left_index=True, right_index=True, how='left'
)

train_idxs = labels[labels.notnull()].index
test_idxs = labels[labels.isnull()].index
train_data = grouped_data.loc[train_idxs]
train_labels = labels.loc[train_idxs]
test_data = grouped_data.loc[test_idxs]

In [43]:
from sklearn.model_selection import TimeSeriesSplit, StratifiedKFold, KFold, GroupKFold
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from mint.mltools.feature import ColumnSelectorPD
from mint.mltools.feature import BorutaPD

### Обучаем 10 lgbm-ов и усредняем

In [86]:
%%time
predictions = []
for i in range(10):
    print(i)
    lgb = LGBMClassifier(n_estimators=1000, reg_alpha=4, 
                         learning_rate=0.03, max_depth=2, random_state=i*777, n_jobs=-1, subsample=1 - i / 100)
    lgb.fit(train_data.values, train_labels.values)
    test_labels = lgb.predict_proba(test_data.values)[:, 1]
    predictions.append(test_labels)
    print(np.mean(test_labels))

0
0.0336546838075
1
0.0336063883323
2
0.0336614730185
3
0.0336389382761
4
0.0336054025023
5
0.0335697290868
6
0.0336817723053
7
0.0336034814096
8
0.0335689431991
9
0.0336774894148
CPU times: user 1h 8min 46s, sys: 2.86 s, total: 1h 8min 48s
Wall time: 8min 58s


In [107]:
pd.DataFrame(np.mean(predictions, axis=0), index=test_idxs).rename_axis(
    {0:'Score'}, axis=1).to_csv('msu_submit_lastt.csv')